<a href="https://colab.research.google.com/github/ashishpatel26/Semantic-Segmentation-Keras-Tensorflow-Example/blob/main/Unet_VGG16_Block_11_class_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.style.use("ggplot")
import itertools
import glob
%matplotlib inline

from tqdm.notebook import trange, tqdm

from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import *
from tensorflow.keras.applications import vgg16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras import backend as K

os.chdir('/content/drive/Shared drives/Deep learning drive/Semantic Segmentation/')

In [8]:
def getImageArr(im):

    img = im.astype(np.float32)

    img[:, :, 0] -= 103.939
    img[:, :, 1] -= 116.779
    img[:, :, 2] -= 123.68

    return img


def getSegmentationArr(seg, nClasses, input_height, input_width):

    seg_labels = np.zeros((input_height, input_width, nClasses))

    for c in range(nClasses):
      seg_labels[:, :, c] = (seg == c).astype(int)

    seg_labels = np.reshape(seg_labels, (-1, nClasses))

    return seg_labels

def imageSegmentationGenerator(images_path, segs_path, batch_size, n_classes, input_height, input_width):

  assert images_path[-1] == '/'
  assert segs_path[-1] == '/'

  images = sorted(glob.glob(images_path + "*.jpg") + 
                  glob.glob(images_path + "*.png") + 
                  glob.glob(images_path + "*.jpeg"))

  segmentations = sorted(glob.glob(segs_path + "*.jpg") + 
                          glob.glob(segs_path + "*.png") + 
                          glob.glob(segs_path + "*.jpeg"))

  zipped = itertools.cycle(zip(images, segmentations))

  while True:
      X = []
      Y = []
      for _ in tqdm(range(batch_size)):
          im, seg = zipped.__next__()
          im = cv2.imread(im, 1)
          seg = cv2.imread(seg, 0)

          assert im.shape[:2] == seg.shape[:2]

          assert im.shape[0] >= input_height and im.shape[1] >= input_width

          xx = random.randint(0, im.shape[0] - input_height)
          yy = random.randint(0, im.shape[1] - input_width)

          im = im[xx:xx + input_height, yy:yy + input_width]
          seg = seg[xx:xx + input_height, yy:yy + input_width]

          X.append(getImageArr(im))
          Y.append(getSegmentationArr(seg,
                                      n_classes,
                                      input_height,
                                      input_width))

      yield np.array(X), np.array(Y)

In [9]:
# !unzip 'dataset1.zip' -d './'
input_height = 320
input_width = 320
n_classes = 11
batch_size = 8

In [4]:
ids = next(os.walk("./dataset1/images_prepped_train/"))[2] # list of names all images in the given path
print("No. of images = ", len(ids))

No. of images =  367


## Data Reading and Train Test

In [12]:
# Manually reading the file and Generate the list of Images
X_train, y_train = imageSegmentationGenerator(images_path = "./dataset1/images_prepped_train/",
                                              segs_path = "./dataset1/annotations_prepped_train/", 
                                              batch_size = 367, 
                                              n_classes=n_classes, 
                                              input_height=input_height, 
                                              input_width=input_width).__next__()
print(X_train.shape, y_train.shape)

X_test, y_test = imageSegmentationGenerator(images_path = "./dataset1/images_prepped_test/",
                                             segs_path = "./dataset1/annotations_prepped_test/",
                                            batch_size = 101, 
                                            n_classes=n_classes, 
                                            input_height=320, 
                                            input_width=320).__next__()
print(X_test.shape, y_test.shape)                          


(367, 320, 320, 3) (367, 102400, 11)



(101, 320, 320, 3) (101, 102400, 11)


In [5]:
# def prepare_X_and_Y(train ,train_mask, h = h, w = w):
#     """
#     Prepare X and y dataset

#     Args:
#         train: (bool): List of Images
#         train_mask: (int): List of Masked Images
#         h: (todo): Height of Images
#         w: (todo): Width of Images
#     """
#   # prepare the zeros array of x and y
#     X_train = np.zeros((len(train), h, w, 3), dtype=np.float32)
#     y_train = np.zeros((len(train_mask), h, w, 1), dtype=np.float32)

#     for n, (img, mimg) in tqdm(enumerate(zip(train, train_mask))):
#       # Load images
#       img = load_img(img)
#       x_img = img_to_array(img)
#       x_img = resize(x_img, (h, w, 3), mode = 'constant', preserve_range = True)
#       # # Load masks
#       mask = img_to_array(load_img(mimg, color_mode = "grayscale"))
#       mask = resize(mask, (h, w, 1), mode = 'constant', preserve_range = True)
#       # Save images
#       X_train[n] = x_img/255.0
#       y_train[n] = mask/255.0

#     print(f"Shape of Images: {X_train.shape}\nShape of Masked Images: {y_train.shape}")  
#     return X_train, y_train

In [24]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
valset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

## Visualize the Example

In [19]:
# # Visualize any randome image along with the mask
# ix = random.randint(0, len(X_train))
# has_mask = y_train_new[ix].max() > 0 # salt indicator

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 15))

# ax1.imshow(X_train[ix, ..., 0], cmap = 'seismic', interpolation = 'bilinear')
# if has_mask: # if salt
#     # draw a boundary(contour) in the original image separating salt and non-salt areas
#     ax1.contour(y_train_new[ix].squeeze(), colors = 'k', linewidths = 5, levels = [0.5])
# ax1.set_title('Segmented')
# ax1.set_axis_off()

# ax2.imshow(y_train_new[ix].squeeze(), cmap = 'gray', interpolation = 'bilinear')
# ax2.set_title('Binary Mask')
# ax2.set_axis_off()

### Unet Model Design

In [25]:
def UNet(nClasses, input_height, input_width):

    assert input_height % 32 == 0
    assert input_width % 32 == 0

    img_input = Input(shape=(input_height, input_width, 3))

    vgg_streamlined = vgg16.VGG16(include_top=False,weights='imagenet', input_tensor=img_input)
    assert isinstance(vgg_streamlined, Model)

    # Exploding path
    o = UpSampling2D((2, 2))(vgg_streamlined.output)
    o = concatenate([vgg_streamlined.get_layer(name="block4_pool").output, o], axis=-1)
    o = Conv2D(512, (3, 3), padding="same")(o)
    o = BatchNormalization()(o)

    o = UpSampling2D((2, 2))(o)
    o = concatenate([vgg_streamlined.get_layer(name="block3_pool").output, o], axis=-1)
    o = Conv2D(256, (3, 3), padding="same")(o)
    o = BatchNormalization()(o)

    o = UpSampling2D((2, 2))(o)
    o = concatenate([vgg_streamlined.get_layer(name="block2_pool").output, o], axis=-1)
    o = Conv2D(128, (3, 3), padding="same")(o)
    o = BatchNormalization()(o)

    o = UpSampling2D((2, 2))(o)
    o = concatenate([vgg_streamlined.get_layer(name="block1_pool").output, o], axis=-1)
    o = Conv2D(64, (3, 3), padding="same")(o)
    o = BatchNormalization()(o)

    # UNet network processing the input mirror magnification 2 times, so the final input and output are reduced by 2 times
    # Here directly upsample and set the original size
    o = UpSampling2D((2, 2))(o)
    o = Conv2D(64, (3, 3), padding="same")(o)
    o = BatchNormalization()(o)

    o = Conv2D(nClasses, (1, 1), padding="same")(o)
    o = BatchNormalization()(o)
    o = Activation("relu")(o)

    o = Reshape((-1, nClasses))(o)
    o = Activation("softmax")(o)

    model = Model(inputs=img_input, outputs=o)
    return model

In [26]:
model = UNet(nClasses=n_classes, input_height=h, input_width=w)
model.compile(loss='categorical_crossentropy', optimizer="adadelta", metrics=['accuracy'])
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 320, 320, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 320, 320, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 320, 320, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 160, 160, 64) 0           block1_conv2[0][0]               
_______________________________________________________________________________________

In [27]:
callbacks = [
    # EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-satellight-image.h5', verbose=1, save_best_only=True, save_weights_only=True),
    TensorBoard(log_dir='./logs')
]

In [28]:
results = model.fit(dataset,
                    batch_size =batch_size,
                    steps_per_epoch= abs(367./batch_size), 
                    epochs=50, 
                    callbacks=callbacks, 
                    validation_data=valset, 
                    use_multiprocessing=True,
                    validation_steps = batch_size,
                    verbose=2)

Epoch 1/50
Instructions for updating:
use `tf.profiler.experimental.stop` instead.

Epoch 00001: val_loss improved from inf to 4.76689, saving model to model-satellight-image.h5
12/45 - 33s - loss: 2.4690 - accuracy: 0.0642 - val_loss: 4.7669 - val_accuracy: 0.0437
